<a href="https://colab.research.google.com/github/aliprg0/Google_Colab/blob/main/FD_9G5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
import glob

In [22]:
clmns = [
    "Open 1","Close 1","Open 2","Close 2","Open 3","Close 3","Open 4","Close 4","Open 5","Close 5","Open 6","Close 6","Open 7","Close 7","Open 8","Close 8","Open 9","Close 9","Open 10","Close 10","Open 11","Close 11","Open 12","Close 12","Open 13","Close 13","Open 14","Close 14","Open 15","Close 15","Open 16","Close 16","Open 17","Close 17","Open 18","Close 18","Open 19","Close 19","Open 20","Close 20","Open 21","Close 21","Open 22","Close 22","Open 23","Close 23","Open 24","Close 24","Open 25","Close 25","Open 26","Close 26","Open 27","Close 27","Open 28","Close 28","Open 29","Close 29","Open 30","Close 30",
    "Open 31","Close 31","Open 32","Close 32","Open 33","Close 33","Open 34","Close 34","Open 35","Close 35","Open 36","Close 36","Open 37","Close 37","Open 38","Close 38","Open 39","Close 39","Open 40","Close 40","Open 41","Close 41","Open 42","Close 42","Open 43","Close 43","Open 44","Close 44","Open 45","Close 45","Open 46","Close 46","Open 47","Close 47","Open 48","Close 48","Open 49","Close 49","Open 50","Close 50","Open 51","Close 51","Open 52","Close 52","Open 53","Close 53","Open 54","Close 54","Open 55","Close 55","Open 56","Close 56","Open 57","Close 57","Open 58","Close 58","Open 59","Close 59","Open 60","Close 60"
    "suggestion"]
  with open("top500.txt","r") as f:
     lines = f.readlines()
     f.close()
  nl = []
  for line in lines:
     nl.append(line.strip())
  return nl
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")

  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")

  with open("yahoo_stocklist.txt","r")as f:
    lines = f.readlines()
    nlines = []
    for line in lines:
       nlines.append(line.strip())
    return nlines
def read_syms_from_txt():
  with open("syms.txt", "r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["yes","no"], axis=1)
  y = data[["yes","no"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-1,1))
    row = scaler.fit_transform(row)
    return row
def process(data):
    data = data.dropna()
    row = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    for i in range(15, data.shape[0]):
        grow = []
        ggrow = []
        gggrow = []
        for x in range(1, 61):
            grow.append([data[i-x, 0], data[i-x, 1],
                        data[i-x, 2], data[i-x, 3]])

        for eeach in grow:
            ggrow.append(scaler(np.array(eeach).reshape(-1, 1)))
        
        for eachh in ggrow:
            eachh = np.array(eachh).reshape(-1,1)
            eachh = np.delete(eachh,1)
            eachh = np.delete(eachh, 1)
            gggrow.append(eachh)


        sugg = "no"
        if data[i][3] > data[i-1][3]:
            sugg = "yes"

        arr = np.array(gggrow).flatten()
        arr = np.append(arr, sugg)
        row.append(arr)

    grow = []
    ggrow = []
    gggrow = []
    arr = []
    
    return np.array(row)

In [48]:
def download_data(symbols,periodd,intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}",end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,interval=intervall, progress=False,show_errors=False)
        if data.empty:
           pass
        else:
           if data.shape[0] > 60:
             data.to_csv(f"/content/data/{symbol}.csv")         
    except:
       print("Error!")
  print(" ")
def each_file_proc(files,now,index):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 40 == 0:
          print(" ")
        address = f"/content/data/{file}"
    
        unattached_dfs.append(process(pd.read_csv(address)))
      
      
     
     if np.array(unattached_dfs[0]).shape[0] == 0:
            print("Null!")
            data = np.array(unattached_dfs[1])
            for z in unattached_dfs[2:]:
               try: 
                  data = np.append(data, np.array(z), axis=0)
               except:
                  pass
     else:
            data = np.array(unattached_dfs[0])
            for z in unattached_dfs[1:]:
               try: 
                  data = np.append(data, np.array(z), axis=0)
               except:
                  pass
        
     unattached_dfs = []
  
     data = pd.DataFrame(data, columns=clmns)
     sugg = data["suggestion"]
     data.drop("suggestion",axis=1,inplace=True)
     sugg = pd.get_dummies(sugg)
     data = pd.concat([data,sugg],axis=1)
     data = data.astype(float)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces):
  pd.options.mode.chained_assignment = None
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  
  index = 0 
  for files in new_files:
     
     each_file_proc(files,now,index)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   data = []
   return xTrain,xTest,yTrain,yTest
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")      

In [49]:
symbols = get_crypto_syms()
#symbols = read_txt_list()
#symbols = read_syms_from_txt()
#symbols = read_top_500()
#symbols = ["EURUSD=X","JPY=X","GBPUSD=X","AUDUSD=X","EURJPY=X","EURHUF=X","EURSEK=X","EURGBP=X","EURCAD=X","IDR=X","THB=X","MYR=X","ZAR=X","RUB=X","CHF=X","CNY=X"]
#symbols = ["EURUSD=X"]
print(f"Symbols : {len(symbols)}")
download_data(symbols,"7d","15m")

Symbols : 1500
Data Folder Removed
 -- 100 -- 200 -- 300 -- 400 -- 500 -- 600 -- 700 -- 800 -- 900 -- 1000 -- 1100 -- 1200 -- 1300 -- 1400 -- 1500 


In [50]:
folder_name = extract_data(100)
to_par(folder_name,10)
xTrain,xTest,yTrain,yTest = make_df(folder_name)

Files In Data : 1419
Processing File:
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40  
41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 

KeyboardInterrupt: ignored

In [51]:
model = Sequential()


model.add(Dense(750, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(750, activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(750, activation='relu'))




opt = tf.keras.optimizers.Adamax()

model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_47 (Dense)            (None, 750)               90750     
                                                                 
 dense_48 (Dense)            (None, 750)               563250    
                                                                 
 dense_49 (Dense)            (None, 750)               563250    
                                                                 
 dense_50 (Dense)            (None, 750)               563250    
                                                                 
 dense_51 (Dense)            (None, 750)               563250    
                                                                 
 dense_52 (Dense)            (None, 750)               563250    
                                                                 
 dense_53 (Dense)            (None, 2)                

In [57]:
model.fit(xTrain,yTrain,epochs=30,batch_size=128,validation_data=(xTest,yTest))

Epoch 1/30
 31/284 [==>...........................] - ETA: 15s - loss: 0.4331 - accuracy: 0.7989

KeyboardInterrupt: ignored

In [ ]:
model.save("CM9G4_9485_C.h5")

In [ ]:
data = yf.download("msft",period="100d",interval="1h")
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
2021-12-07 09:30:00-05:00,331.640015,335.269989,330.320007,334.290009,334.290009,7257768
2021-12-07 10:30:00-05:00,334.299988,335.799988,334.299988,334.910004,334.910004,3585429
2021-12-07 11:30:00-05:00,334.904999,335.779999,334.670013,334.809998,334.809998,2692466
2021-12-07 12:30:00-05:00,334.829987,335.779999,334.640015,334.834991,334.834991,2345770
2021-12-07 13:30:00-05:00,334.839996,335.679993,333.975006,334.239990,334.239990,2258992
...,...,...,...,...,...,...
2022-04-29 12:30:00-04:00,283.700012,284.480011,282.899994,283.709991,283.709991,1932710
2022-04-29 13:30:00-04:00,283.679993,284.119995,281.230011,281.579987,281.579987,2134130
2022-04-29 14:30:00-04:00,281.540009,281.959991,279.970001,280.029999,280.029999,3713287
2022-04-29 15:30:00-04:00,280.010010,280.649994,276.500000,277.529999,277.529999,7118487


In [ ]:
def process_for_prediction(data):
        i = -1
        row = []
        if len(pd.DataFrame(data).columns) == 7:
          data = data.iloc[: , 1:]        
        data = np.array(data)
        grow = []
      
        s12 = [ 
            data[i-12][0], data[i-12][1], data[i-12][2], data[i-12][3]
        ]
        s11 = [
            data[i-11][0], data[i-11][1], data[i-11][2], data[i-11][3]
        ]
        s10 = [
            data[i-10][0], data[i-10][1], data[i-10][2], data[i-10][3]
        ]
        s9 = [
            data[i-9][0], data[i-9][1], data[i-9][2], data[i-9][3]
        ]
        s8 = [
            data[i-8][0], data[i-8][1], data[i-8][2], data[i-8][3]
        ]
        s7 = [
            data[i-7][0], data[i-7][1], data[i-7][2], data[i-7][3]
        ]
        s6 = [
            data[i-6][0], data[i-6][1], data[i-6][2], data[i-6][3]
        ]
        s5 = [
            data[i-5][0], data[i-5][1], data[i-5][2], data[i-5][3]
        ]   
        s4 = [
            data[i-4][0], data[i-4][1], data[i-4][2], data[i-4][3]
        ]
        s3 = [
            data[i-3][0], data[i-3][1], data[i-3][2], data[i-3][3]
        ]
        s2 = [
            data[i-2][0], data[i-2][1], data[i-2][2], data[i-2][3]
        ]
        s1 = [
            data[i-1][0], data[i-1][1], data[i-1][2], data[i-1][3]
        ]
        
        s13 = [
            data[i-13][0], data[i-13][1], data[i-13][2], data[i-13][3]
        ]
        s14 = [
            data[i-14][0], data[i-14][1], data[i-14][2], data[i-14][3]
        ]
        s15 = [
            data[i-15][0], data[i-15][1], data[i-15][2], data[i-15][3]
        ]   
        s16 = [
            data[i-16][0], data[i-16][1], data[i-16][2], data[i-16][3]
        ]
        s17 = [
            data[i-17][0], data[i-17][1], data[i-17][2], data[i-17][3]
        ]
        s18 = [
            data[i-18][0], data[i-18][1], data[i-18][2], data[i-18][3]
        ]   
        s19 = [
            data[i-19][0], data[i-19][1], data[i-19][2], data[i-19][3]
        ]
        s20 = [
            data[i-20][0], data[i-20][1], data[i-20][2], data[i-20][3]
        ]
        s21 = [
            data[i-21][0], data[i-21][1], data[i-21][2], data[i-21][3]
        ]   
        s22 = [
            data[i-22][0], data[i-22][1], data[i-22][2], data[i-22][3]
        ]
        s23 = [
            data[i-23][0], data[i-23][1], data[i-23][2], data[i-23][3]
        ]
        s24 = [
            data[i-24][0], data[i-24][1], data[i-24][2], data[i-24][3]
        ]
        s25 = [
            data[i-25][0], data[i-25][1], data[i-25][2], data[i-25][3]
        ]
        s26 = [
            data[i-26][0], data[i-26][1], data[i-26][2], data[i-26][3]
        ]   
        s27 = [
            data[i-27][0], data[i-27][1], data[i-27][2], data[i-27][3]
        ]
        s28 = [
            data[i-28][0], data[i-28][1], data[i-28][2], data[i-28][3]
        ]   
        s29 = [
            data[i-29][0], data[i-29][1], data[i-29][2], data[i-29][3]
        ]   
        s30 = [
            data[i-30][0], data[i-30][1], data[i-30][2], data[i-30][3]]
        
        s31 = [
            data[i-31][0], data[i-31][1], data[i-31][2], data[i-31][3]]
        s32 = [
            data[i-32][0], data[i-32][1], data[i-32][2], data[i-32][3]]
        s33 = [
            data[i-33][0], data[i-33][1], data[i-33][2], data[i-33][3]]
        s34 = [
            data[i-34][0], data[i-34][1], data[i-34][2], data[i-34][3]]
        s35 = [
            data[i-35][0], data[i-35][1], data[i-35][2], data[i-35][3]]
        s36 = [
            data[i-36][0], data[i-36][1], data[i-36][2], data[i-36][3]]
        s37 = [
            data[i-37][0], data[i-37][1], data[i-37][2], data[i-37][3]]
        s38 = [
            data[i-38][0], data[i-38][1], data[i-38][2], data[i-38][3]]
        s39 = [
            data[i-39][0], data[i-39][1], data[i-39][2], data[i-39][3]]
        s40 = [
            data[i-40][0], data[i-40][1], data[i-40][2], data[i-40][3]]
        s41 = [
            data[i-41][0], data[i-41][1], data[i-41][2], data[i-41][3]]
        s42 = [
            
            data[i-42][0], data[i-42][1], data[i-42][2], data[i-42][3]]
        s43 = [
            data[i-43][0], data[i-43][1], data[i-43][2], data[i-43][3]]
        s44 = [
            data[i-44][0], data[i-44][1], data[i-44][2], data[i-44][3]]
        s45 = [
            data[i-45][0], data[i-45][1], data[i-45][2], data[i-45][3]]
        s46 = [
            data[i-46][0], data[i-46][1], data[i-46][2], data[i-46][3]]
        s47 = [
            data[i-47][0], data[i-47][1], data[i-47][2], data[i-47][3]]
        s48 = [
            data[i-48][0], data[i-48][1], data[i-48][2], data[i-48][3]]
        s49 = [
            data[i-49][0], data[i-49][1], data[i-49][2], data[i-49][3]]
        s50 = [
            data[i-50][0], data[i-50][1], data[i-50][2], data[i-50][3]]


        s51 = [
            data[i-51][0], data[i-51][1], data[i-51][2], data[i-51][3]]
        s52 = [ 
            data[i-52][0], data[i-52][1], data[i-52][2], data[i-52][3]]
        s53 = [
            data[i-53][0], data[i-53][1], data[i-53][2], data[i-53][3]]
        s54 = [
            data[i-54][0], data[i-54][1], data[i-54][2], data[i-54][3]]
        s55 = [
            data[i-55][0], data[i-55][1], data[i-55][2], data[i-55][3]]
        s56 = [
            data[i-56][0], data[i-56][1], data[i-56][2], data[i-56][3]]
        s57 = [
            data[i-57][0], data[i-57][1], data[i-57][2], data[i-57][3]]
        s58 = [
            data[i-58][0], data[i-58][1], data[i-58][2], data[i-58][3]]
        s59 = [
            data[i-59][0], data[i-59][1], data[i-59][2], data[i-59][3]]
        s60 = [
            data[i-60][0], data[i-60][1], data[i-60][2], data[i-60][3]]   

        s1 = scaler(np.array(s1).reshape(-1,1))
        s2 = scaler(np.array(s2).reshape(-1,1))
        s3 = scaler(np.array(s3).reshape(-1,1))
        s4 = scaler(np.array(s4).reshape(-1,1))
        s5 = scaler(np.array(s5).reshape(-1,1))
        s6 = scaler(np.array(s6).reshape(-1,1))
        s7 = scaler(np.array(s7).reshape(-1,1))
        s8 = scaler(np.array(s8).reshape(-1,1))
        s9 = scaler(np.array(s9).reshape(-1,1))
        s10 = scaler(np.array(s10).reshape(-1,1))
        s11 = scaler(np.array(s11).reshape(-1,1))
        s12 = scaler(np.array(s12).reshape(-1,1))
        s13 = scaler(np.array(s13).reshape(-1,1))
        s14 = scaler(np.array(s14).reshape(-1,1))
        s15 = scaler(np.array(s15).reshape(-1,1))
        s16 = scaler(np.array(s16).reshape(-1,1))
        s17 = scaler(np.array(s17).reshape(-1,1))
        s18 = scaler(np.array(s18).reshape(-1,1))
        s19 = scaler(np.array(s19).reshape(-1,1))
        s20 = scaler(np.array(s20).reshape(-1,1))
        s21 = scaler(np.array(s21).reshape(-1,1))
        s22 = scaler(np.array(s22).reshape(-1,1))
        s23 = scaler(np.array(s23).reshape(-1,1))
        s24 = scaler(np.array(s24).reshape(-1,1))
        s25 = scaler(np.array(s25).reshape(-1,1))
        s26 = scaler(np.array(s26).reshape(-1,1))
        s27 = scaler(np.array(s27).reshape(-1,1))
        s28 = scaler(np.array(s28).reshape(-1,1))
        s29 = scaler(np.array(s29).reshape(-1,1))
        s30 = scaler(np.array(s30).reshape(-1,1))
        s31 = scaler(np.array(s31).reshape(-1,1))
        s32 = scaler(np.array(s32).reshape(-1,1))
        s33 = scaler(np.array(s33).reshape(-1,1))
        s34 = scaler(np.array(s34).reshape(-1,1))
        s35 = scaler(np.array(s35).reshape(-1,1))
        s36 = scaler(np.array(s36).reshape(-1,1))
        s37 = scaler(np.array(s37).reshape(-1,1))
        s38 = scaler(np.array(s38).reshape(-1,1))
        s39 = scaler(np.array(s39).reshape(-1,1))
        s40 = scaler(np.array(s40).reshape(-1,1))
        s41 = scaler(np.array(s41).reshape(-1,1))
        s42 = scaler(np.array(s42).reshape(-1,1))
        s43 = scaler(np.array(s43).reshape(-1,1))
        s44 = scaler(np.array(s44).reshape(-1,1))
        s45 = scaler(np.array(s45).reshape(-1,1))
        s46 = scaler(np.array(s46).reshape(-1,1))
        s47 = scaler(np.array(s47).reshape(-1,1))
        s48 = scaler(np.array(s48).reshape(-1,1))
        s49 = scaler(np.array(s49).reshape(-1,1))
        s50 = scaler(np.array(s50).reshape(-1,1))
        s51 = scaler(np.array(s51).reshape(-1,1))
        s52 = scaler(np.array(s52).reshape(-1,1))
        s53 = scaler(np.array(s53).reshape(-1,1))
        s54 = scaler(np.array(s54).reshape(-1,1))
        s55 = scaler(np.array(s55).reshape(-1,1))
        s56 = scaler(np.array(s56).reshape(-1,1))
        s57 = scaler(np.array(s57).reshape(-1,1))
        s58 = scaler(np.array(s58).reshape(-1,1))
        s59 = scaler(np.array(s59).reshape(-1,1))
        s60 = scaler(np.array(s60).reshape(-1,1))

        grow = np.vstack((s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11, s12, s13, s14, s15, s16, s17, s18, s19, s20, s21, s22, s23, s24, s25, s26, s27, s28, s29, s30, s31, s32, s33, s34, s35, s36, s37, s38, s39, s40, s41, s42, s43, s44, s45, s46, s47, s48, s49, s50, s51, s52, s53, s54, s55, s56, s57, s58, s59, s60))

        arr = np.array(grow).flatten()
        row.append(arr)

        return np.array(row)
def make_prediction(symbol,timeframe):
  if timeframe == "5m":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))    
  if timeframe == "15m":
    data = yf.download(symbol,period="30d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1h":
    data = yf.download(symbol,period="100d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1d":
    data = yf.download(symbol,period="max",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1wk":
    data = yf.download(symbol,period="max",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1mo":
    data = yf.download(symbol,period="max",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "1m":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "2m":
    data = yf.download(symbol,period="max",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "30m":
    data = yf.download(symbol,period="1d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "5d":
    data = yf.download(symbol,period="5mo",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "90m":
    data = yf.download(symbol,period="5d",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  
  if timeframe == "3mo":
    data = yf.download(symbol,period="max",interval=timeframe)
    lst = [str(i) for i in data.index.values]
    ddata = process_for_prediction(data)
    return(model.predict(ddata))  

In [ ]:
make_prediction("btc-usd","15m")

[*********************100%***********************]  1 of 1 completed


array([[9.9975014e-01, 2.8190017e-04]], dtype=float32)

In [ ]:
def process_for_prediction(data):
        i = -1
        row = []
        data.drop("symbol",axis=1,inplace=True)   
        data = np.array(data)
        grow = []
      
        s12 = [ 
            data[i-12][0], data[i-12][1], data[i-12][2], data[i-12][3]
        ]
        s11 = [
            data[i-11][0], data[i-11][1], data[i-11][2], data[i-11][3]
        ]
        s10 = [
            data[i-10][0], data[i-10][1], data[i-10][2], data[i-10][3]
        ]
        s9 = [
            data[i-9][0], data[i-9][1], data[i-9][2], data[i-9][3]
        ]
        s8 = [
            data[i-8][0], data[i-8][1], data[i-8][2], data[i-8][3]
        ]
        s7 = [
            data[i-7][0], data[i-7][1], data[i-7][2], data[i-7][3]
        ]
        s6 = [
            data[i-6][0], data[i-6][1], data[i-6][2], data[i-6][3]
        ]
        s5 = [
            data[i-5][0], data[i-5][1], data[i-5][2], data[i-5][3]
        ]   
        s4 = [
            data[i-4][0], data[i-4][1], data[i-4][2], data[i-4][3]
        ]
        s3 = [
            data[i-3][0], data[i-3][1], data[i-3][2], data[i-3][3]
        ]
        s2 = [
            data[i-2][0], data[i-2][1], data[i-2][2], data[i-2][3]
        ]
        s1 = [
            data[i-1][0], data[i-1][1], data[i-1][2], data[i-1][3]
        ]
        
        s13 = [
            data[i-13][0], data[i-13][1], data[i-13][2], data[i-13][3]
        ]
        s14 = [
            data[i-14][0], data[i-14][1], data[i-14][2], data[i-14][3]
        ]
        s15 = [
            data[i-15][0], data[i-15][1], data[i-15][2], data[i-15][3]
        ]   
        s16 = [
            data[i-16][0], data[i-16][1], data[i-16][2], data[i-16][3]
        ]
        s17 = [
            data[i-17][0], data[i-17][1], data[i-17][2], data[i-17][3]
        ]
        s18 = [
            data[i-18][0], data[i-18][1], data[i-18][2], data[i-18][3]
        ]   
        s19 = [
            data[i-19][0], data[i-19][1], data[i-19][2], data[i-19][3]
        ]
        s20 = [
            data[i-20][0], data[i-20][1], data[i-20][2], data[i-20][3]
        ]
        s21 = [
            data[i-21][0], data[i-21][1], data[i-21][2], data[i-21][3]
        ]   
        s22 = [
            data[i-22][0], data[i-22][1], data[i-22][2], data[i-22][3]
        ]
        s23 = [
            data[i-23][0], data[i-23][1], data[i-23][2], data[i-23][3]
        ]
        s24 = [
            data[i-24][0], data[i-24][1], data[i-24][2], data[i-24][3]
        ]
        s25 = [
            data[i-25][0], data[i-25][1], data[i-25][2], data[i-25][3]
        ]
        s26 = [
            data[i-26][0], data[i-26][1], data[i-26][2], data[i-26][3]
        ]   
        s27 = [
            data[i-27][0], data[i-27][1], data[i-27][2], data[i-27][3]
        ]
        s28 = [
            data[i-28][0], data[i-28][1], data[i-28][2], data[i-28][3]
        ]   
        s29 = [
            data[i-29][0], data[i-29][1], data[i-29][2], data[i-29][3]
        ]   
        s30 = [
            data[i-30][0], data[i-30][1], data[i-30][2], data[i-30][3]]
        
        s31 = [
            data[i-31][0], data[i-31][1], data[i-31][2], data[i-31][3]]
        s32 = [
            data[i-32][0], data[i-32][1], data[i-32][2], data[i-32][3]]
        s33 = [
            data[i-33][0], data[i-33][1], data[i-33][2], data[i-33][3]]
        s34 = [
            data[i-34][0], data[i-34][1], data[i-34][2], data[i-34][3]]
        s35 = [
            data[i-35][0], data[i-35][1], data[i-35][2], data[i-35][3]]
        s36 = [
            data[i-36][0], data[i-36][1], data[i-36][2], data[i-36][3]]
        s37 = [
            data[i-37][0], data[i-37][1], data[i-37][2], data[i-37][3]]
        s38 = [
            data[i-38][0], data[i-38][1], data[i-38][2], data[i-38][3]]
        s39 = [
            data[i-39][0], data[i-39][1], data[i-39][2], data[i-39][3]]
        s40 = [
            data[i-40][0], data[i-40][1], data[i-40][2], data[i-40][3]]
        s41 = [
            data[i-41][0], data[i-41][1], data[i-41][2], data[i-41][3]]
        s42 = [
            
            data[i-42][0], data[i-42][1], data[i-42][2], data[i-42][3]]
        s43 = [
            data[i-43][0], data[i-43][1], data[i-43][2], data[i-43][3]]
        s44 = [
            data[i-44][0], data[i-44][1], data[i-44][2], data[i-44][3]]
        s45 = [
            data[i-45][0], data[i-45][1], data[i-45][2], data[i-45][3]]
        s46 = [
            data[i-46][0], data[i-46][1], data[i-46][2], data[i-46][3]]
        s47 = [
            data[i-47][0], data[i-47][1], data[i-47][2], data[i-47][3]]
        s48 = [
            data[i-48][0], data[i-48][1], data[i-48][2], data[i-48][3]]
        s49 = [
            data[i-49][0], data[i-49][1], data[i-49][2], data[i-49][3]]
        s50 = [
            data[i-50][0], data[i-50][1], data[i-50][2], data[i-50][3]]

        s51 = [
            data[i-51][0], data[i-51][1], data[i-51][2], data[i-51][3]]
        s52 = [ 
            data[i-52][0], data[i-52][1], data[i-52][2], data[i-52][3]]
        s53 = [
            data[i-53][0], data[i-53][1], data[i-53][2], data[i-53][3]]
        s54 = [
            data[i-54][0], data[i-54][1], data[i-54][2], data[i-54][3]]
        s55 = [
            data[i-55][0], data[i-55][1], data[i-55][2], data[i-55][3]]
        s56 = [
            data[i-56][0], data[i-56][1], data[i-56][2], data[i-56][3]]
        s57 = [
            data[i-57][0], data[i-57][1], data[i-57][2], data[i-57][3]]
        s58 = [
            data[i-58][0], data[i-58][1], data[i-58][2], data[i-58][3]]
        s59 = [
            data[i-59][0], data[i-59][1], data[i-59][2], data[i-59][3]]
        s60 = [
            data[i-60][0], data[i-60][1], data[i-60][2], data[i-60][3]]   

        s1 = scaler(np.array(s1).reshape(-1,1))
        s2 = scaler(np.array(s2).reshape(-1,1))
        s3 = scaler(np.array(s3).reshape(-1,1))
        s4 = scaler(np.array(s4).reshape(-1,1))
        s5 = scaler(np.array(s5).reshape(-1,1))
        s6 = scaler(np.array(s6).reshape(-1,1))
        s7 = scaler(np.array(s7).reshape(-1,1))
        s8 = scaler(np.array(s8).reshape(-1,1))
        s9 = scaler(np.array(s9).reshape(-1,1))
        s10 = scaler(np.array(s10).reshape(-1,1))
        s11 = scaler(np.array(s11).reshape(-1,1))
        s12 = scaler(np.array(s12).reshape(-1,1))
        s13 = scaler(np.array(s13).reshape(-1,1))
        s14 = scaler(np.array(s14).reshape(-1,1))
        s15 = scaler(np.array(s15).reshape(-1,1))
        s16 = scaler(np.array(s16).reshape(-1,1))
        s17 = scaler(np.array(s17).reshape(-1,1))
        s18 = scaler(np.array(s18).reshape(-1,1))
        s19 = scaler(np.array(s19).reshape(-1,1))
        s20 = scaler(np.array(s20).reshape(-1,1))
        s21 = scaler(np.array(s21).reshape(-1,1))
        s22 = scaler(np.array(s22).reshape(-1,1))
        s23 = scaler(np.array(s23).reshape(-1,1))
        s24 = scaler(np.array(s24).reshape(-1,1))
        s25 = scaler(np.array(s25).reshape(-1,1))
        s26 = scaler(np.array(s26).reshape(-1,1))
        s27 = scaler(np.array(s27).reshape(-1,1))
        s28 = scaler(np.array(s28).reshape(-1,1))
        s29 = scaler(np.array(s29).reshape(-1,1))
        s30 = scaler(np.array(s30).reshape(-1,1))
        s31 = scaler(np.array(s31).reshape(-1,1))
        s32 = scaler(np.array(s32).reshape(-1,1))
        s33 = scaler(np.array(s33).reshape(-1,1))
        s34 = scaler(np.array(s34).reshape(-1,1))
        s35 = scaler(np.array(s35).reshape(-1,1))
        s36 = scaler(np.array(s36).reshape(-1,1))
        s37 = scaler(np.array(s37).reshape(-1,1))
        s38 = scaler(np.array(s38).reshape(-1,1))
        s39 = scaler(np.array(s39).reshape(-1,1))
        s40 = scaler(np.array(s40).reshape(-1,1))
        s41 = scaler(np.array(s41).reshape(-1,1))
        s42 = scaler(np.array(s42).reshape(-1,1))
        s43 = scaler(np.array(s43).reshape(-1,1))
        s44 = scaler(np.array(s44).reshape(-1,1))
        s45 = scaler(np.array(s45).reshape(-1,1))
        s46 = scaler(np.array(s46).reshape(-1,1))
        s47 = scaler(np.array(s47).reshape(-1,1))
        s48 = scaler(np.array(s48).reshape(-1,1))
        s49 = scaler(np.array(s49).reshape(-1,1))
        s50 = scaler(np.array(s50).reshape(-1,1))
        s51 = scaler(np.array(s51).reshape(-1,1))
        s52 = scaler(np.array(s52).reshape(-1,1))
        s53 = scaler(np.array(s53).reshape(-1,1))
        s54 = scaler(np.array(s54).reshape(-1,1))
        s55 = scaler(np.array(s55).reshape(-1,1))
        s56 = scaler(np.array(s56).reshape(-1,1))
        s57 = scaler(np.array(s57).reshape(-1,1))
        s58 = scaler(np.array(s58).reshape(-1,1))
        s59 = scaler(np.array(s59).reshape(-1,1))
        s60 = scaler(np.array(s60).reshape(-1,1))


        grow = np.vstack((s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11, s12, s13, s14, s15, s16, s17, s18, s19, s20, s21, s22, s23, s24, s25, s26, s27, s28, s29, s30, s31, s32, s33, s34, s35, s36, s37, s38, s39, s40, s41, s42, s43, s44, s45, s46, s47, s48, s49, s50, s51, s52, s53, s54, s55, s56, s57, s58, s59, s60))

        arr = np.array(grow).flatten()
        row.append(arr)

        return np.array(row)
def make_prediction(symbol,timeframe):
   tv = TvDatafeed()
   data = tv.get_hist(symbol='btcUSD',exchange='bitstamp',interval=Interval.in_15_minute,n_bars=100)
   return model.predict(process_for_prediction(data))

In [ ]:
make_prediction("s","x")

array([[0.8006747 , 0.20669705]], dtype=float32)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
yf.download("trx",period="1d",interval="5m")

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-04-29 09:30:00-04:00,0.298000,0.302700,0.290000,0.300849,0.300849,86764
2022-04-29 09:35:00-04:00,0.302490,0.302600,0.301201,0.302400,0.302400,6708
2022-04-29 09:40:00-04:00,0.300000,0.301200,0.300000,0.301200,0.301200,1359
2022-04-29 09:45:00-04:00,0.300101,0.302300,0.300000,0.302300,0.302300,12800
2022-04-29 09:50:00-04:00,0.302000,0.302200,0.302000,0.302200,0.302200,1306
2022-04-29 09:55:00-04:00,0.302199,0.302199,0.302099,0.302099,0.302099,4001
2022-04-29 10:05:00-04:00,0.301900,0.301900,0.301500,0.301500,0.301500,1601
2022-04-29 10:10:00-04:00,0.301500,0.301502,0.301500,0.301500,0.301500,9496
2022-04-29 10:15:00-04:00,0.301500,0.301500,0.301500,0.301500,0.301500,1707
